In [2]:
!pip install mediapipe

  Using cached mediapipe-0.8.11-cp38-cp38-win_amd64.whl (49.0 MB)Requirement already satisfied: python-dateutil>=2.1 in d:\anaconda\lib\site-packages (from matplotlib->mediapipe) (2.8.1)

  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)


In [40]:

import cv2 
import sklearn
import numpy 
import os 
import mediapipe 

from essentials import get_feed,get_dots,get_dot

from sklearn.model_selection import train_test_split 




In [41]:
holistic_model = mediapipe.solutions.holistic
draw_model = mediapipe.solutions.drawing_utils 

In [44]:
holistic_model.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

#### testing if everything works fine

In [45]:
vis = cv2.VideoCapture(0)
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as h_model:
    while vis.isOpened():
        k, image = vis.read()

        image, out = get_feed(image, h_model)
        print(out)
        
        get_dots(image,out)
        cv2.imshow('screen',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    vis.release()
    cv2.destroyAllWindows() 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [46]:
vis.release()

cv2.destroyAllWindows() 

#### creating paths

In [48]:
### DON"T RUN

address = os.path.join('Sign_data') 

signs = numpy.array(['salute','ok' ,'love you'])

# videos,fps,home =30,30,30
videos,fps,home =2,30,30

In [49]:
for i in signs: 
    for j in range(videos):
        try: 
            os.makedirs(os.path.join(address, i, str(j)))
        except:
            pass

### Data collection starts

In [50]:


vis = cv2.VideoCapture(0)

with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as h_model:

    for action in signs:

        for sequence in range(videos):
            for x in range(fps):
                k, image = vis.read()
                image, out = get_feed(image, h_model)
                get_dots(image, out)
                if x == 0: 
                    cv2.putText(image, 'WAIT', (299,199), 
                               cv2.FONT_HERSHEY_PLAIN , 4, (1,255, 1), 2, cv2.LINE_AA)
                    cv2.putText(image, str(action)+" : "+ str(sequence), (5,20), 
                               cv2.FONT_HERSHEY_PLAIN , 1, (1, 1, 266), 1, cv2.LINE_AA)
                    cv2.imshow('screen', image)
                    cv2.waitKey(555)
                else: 
                    cv2.putText(image, str(action)+" : "+ str(sequence), (5,20), cv2.FONT_HERSHEY_PLAIN , 1, (1,1 , 266), 1, cv2.LINE_AA)
                    cv2.imshow('screen', image)
                dots = get_dot(out)
                npy_path = os.path.join(address, action, str(sequence), str(x))
                numpy.save(npy_path, dots)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    vis.release()
    cv2.destroyAllWindows()

In [17]:
vis.release()
cv2.destroyAllWindows()

In [30]:
enc = {l:n for n, l in enumerate(signs)}

In [31]:
enc


{'salute': 0, 'ok': 1, 'love you': 2}

In [32]:
seq, numbers = [], []
for action in signs:
    for s in range(videos):
        lst = []
        for x in range(fps):
            r = numpy.load(os.path.join(address, action, str(s), "{}.npy".format(x)))
            lst.append(r)
        seq.append(lst)
        numbers.append(enc[action])

In [33]:
numbers

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]

In [34]:
X = numpy.array(seq)

In [35]:
X.shape

(15, 30, 1662)

In [36]:
X.shape

(15, 30, 1662)

In [23]:
y = encode(numbers).astype(int)

In [24]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.09,random_state=0)

In [26]:
y_test.shape

(22, 8)

In [38]:
r

array([ 0.53885943,  0.55004472, -1.23177278, ...,  0.556871  ,
        0.72811037, -0.04376552])

In [39]:
npy_path

'newdata\\love you\\4\\29'